In [ ]:
%load_ext autoreload
%autoreload 2

import dill
import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, BLENDER_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import find_ball_initial_velocity, PITCH_START_POSITION

from iiwa_batter.swing_simulator import setup_simulator, run_swing_simulation, reset_systems
from iiwa_batter.plotting import plot_joint_velocity

from iiwa_batter.export_for_blender import export_keyframes

In [ ]:
meshcat = StartMeshcat()

In [ ]:
# Demonstration of the iiwa14 320 Nm joint able to spin up very fast

simulator, diagram = setup_simulator(
    torque_trajectory={0: np.array([320, 0, 0, 0, 0, 0, 0])},
    model_urdf="iiwa14_joint_1_only",
    dt=PITCH_DT, 
    meshcat=meshcat, 
    add_contact=False, 
)

status_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=1,
    # MUST MATCH THE URDF!!!
    initial_joint_positions=np.array([0, np.pi/2, 0, 0, 0, -np.pi/2, np.pi]),
    initial_joint_velocities=np.array([0]*NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state=True,
    check_dt=BLENDER_DT,
)

export_keyframes(status_dict, "iiwa14_torque_demo_no_cap")

In [ ]:
# Demonstration of the iiwa14 320 Nm joint getting speed capped
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]

simulator, diagram = setup_simulator(
    torque_trajectory={0: np.array([200, 0, 0, 0, 0, 0, 0])}, 
    model_urdf="iiwa14_joint_1_only",
    dt=PITCH_DT, 
    meshcat=meshcat, 
    add_contact=False, 
    robot_constraints=robot_constraints)

results_dict = run_swing_simulation(
    simulator=simulator,
    diagram=diagram,
    start_time=0,
    end_time=0.2,
    # MUST MATCH THE URDF!!!
    initial_joint_positions=np.array([0, np.pi/2, 0, 0, 0, -np.pi/2, np.pi]),
    initial_joint_velocities=np.array([0]*NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=np.zeros(3),
    meshcat=meshcat,
    record_state = True
)

plot_joint_velocity(results_dict["state"], robot_constraints, f"{PACKAGE_ROOT}/../notebook_images/capped_velocity.png")



In [ ]:
# Demonstration of the iiwa14 40 Nm joint getting speed capped

In [ ]:
# Demonstration of searching for initial positions
robot_constraints = JOINT_CONSTRAINTS["iiwa14"]
simulator, diagram = setup_simulator(torque_trajectory={}, model_urdf="iiwa14", dt=PITCH_DT, add_contact=True, meshcat=meshcat)
import time

from iiwa_batter.swing_optimization.stochastic_gradient_descent import find_initial_positions
while True:
    initial_joint_positions = find_initial_positions(simulator, diagram, robot_constraints, 1)

    reset_systems(diagram)

    run_swing_simulation(
        simulator=simulator,
        diagram=diagram,
        start_time=0,
        end_time=2*CONTACT_DT,
        initial_joint_positions=np.array([0] * NUM_JOINTS),
        initial_joint_velocities=np.array([0] * NUM_JOINTS),
        initial_ball_position=PITCH_START_POSITION,
        initial_ball_velocity=np.zeros(3),
        meshcat=meshcat,
    )
    time.sleep(2)